In [10]:
import pymysql
import mariadb
import sys

In [6]:
db = pymysql.connect(host='127.0.0.1',user='root',password='540210',db='TW_StkMkt')

In [7]:
cur=db.cursor()

In [8]:
cur.execute('create table StkInfo(Code int, )')

1

In [9]:
Data=[i for i in cur.fetchall()]

In [10]:
Data

[(1, 'test', '100', '40')]

In [11]:
cur.close()

In [12]:
db.close()

In [1]:
def find_code(Names):
    code = []
    for Name in Names:
        if re.match(r'\d{4,6}[a-zA-Z]|\d{4,6}', Name):
            code.append(re.findall(r'\d{4,6}[a-zA-Z]|\d{4,6}', Name))
        else:
            code.append('No matches')
    return [i[0] for i in code]

In [2]:
def find_name(Names):
    name = []
    for Name in Names:   
        name.append(Name.split(' ')[1])
    return [i for i in name]

# Scrape Data

In [3]:
import numpy as np
import requests
import pandas as pd
import datetime
import re
import time

In [4]:
Stk_base = pd.read_excel('C:/Users/Admin/Desktop/TW_StkList.xlsx')

In [5]:
Stk_base['Code'] = find_code(Stk_base['Name'])
Stk_base['Name'] = find_name(Stk_base['Name'])

Stk_base

,Name,Ind,Code
0,麗正,Semi-conductor,2302
1,光罩,Semi-conductor,2338
2,菱生,Semi-conductor,2369
3,統懋,Semi-conductor,2434
4,聯發科,Semi-conductor,2454
...,...,...,...
959,中保科,Others,9917
960,泰銘,Others,9927
961,成霖,Others,9934
962,裕融,Others,9941


In [6]:
#   http://www.twse.com.tw/exchangeReport/STOCK_DAY?date=20180817&stockNo=2330  取一個月的股價與成交量
def get_stock_history(date, stock_no):
    quotes = []
    url = 'http://www.twse.com.tw/exchangeReport/STOCK_DAY?date=%s&stockNo=%s' % ( date, stock_no)
    r = requests.get(url)
    data = r.json()
    return transform(data['data'])  #進行資料格式轉換

def transform_date(date):
        y, m, d = date.split('/')
        return str(int(y)+1911) + '/' + m  + '/' + d  #民國轉西元
    
def transform_data(data):
    data[0] = datetime.datetime.strptime(transform_date(data[0]), '%Y/%m/%d')
    data[1] = int(data[1].replace(',', ''))  #把千進位的逗點去除
    data[2] = int(data[2].replace(',', ''))
    data[3] = float(data[3].replace(',', ''))
    data[4] = float(data[4].replace(',', ''))
    data[5] = float(data[5].replace(',', ''))
    data[6] = float(data[6].replace(',', ''))
    data[7] = float(0.0 if data[7].replace(',', '') == 'X0.00' else data[7].replace(',', ''))  # +/-/X表示漲/跌/不比價
    data[8] = int(data[8].replace(',', ''))
    return data

def transform(data):
    return [transform_data(d) for d in data]

def create_df(date,stock_no):
    s = pd.DataFrame(get_stock_history(date, stock_no))
    s.columns = ['date', 'shares', 'amount', 'open', 'high', 'low', 'close', 'change', 'turnover']
                #"日期","成交股數","成交金額","開盤價","最高價","最低價","收盤價","漲跌價差","成交筆數" 
    stock = []
    for i in range(len(s)):
        stock.append(stock_no)
    s['stockno'] = pd.Series(stock ,index=s.index)  #新增股票代碼欄，之後所有股票進入資料表才能知道是哪一張股票
    datelist = []
    for i in range(len(s)):
        datelist.append(s['date'][i])
    s.index = datelist  #索引值改成日期
    s2 = s.drop(['date'],axis = 1)  #刪除日期欄位
    mlist = []
    for item in s2.index:
        mlist.append(item.month)
    s2['month'] = mlist  #新增月份欄位
    return s2

In [ ]:
Months = pd.date_range('2010-01-01','2020-10-31', freq='MS').strftime("%Y%m").tolist()
listDji = Stk_base['Code']
Stk_Data = pd.DataFrame()
for j in Months:  
    for i in range(len(listDji)):
        print('Processing---Stock: %s, Month: %s'%(listDji[i],j))
        try:
            result = create_df(j+'01', listDji[i])
            Stk_Data.append(result)
            
            time.sleep(3)
        except:
            time.sleep(1)
            next
    Stk_Data.to_csv('stkdata'+j+'.csv',encoding=utf-8-sig)
    

Processing---Stock: 2302, Month: 201001
Processing---Stock: 2338, Month: 201001
Processing---Stock: 2369, Month: 201001
Processing---Stock: 2434, Month: 201001
Processing---Stock: 2454, Month: 201001
Processing---Stock: 3016, Month: 201001
Processing---Stock: 3094, Month: 201001
Processing---Stock: 3530, Month: 201001
Processing---Stock: 3588, Month: 201001


In [ ]:
#Financial Report
#https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=2330&SYEAR=2019&SSEASON=2&REPORT_ID=C 

In [ ]:
print(result.groupby('month').close.count())  #每個月幾個營業日
print(result.groupby('month').shares.sum())  #每個月累計成交股數